In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = "/content/drive/MyDrive/iot_23_datasets_small/opt/Malware-Project/BigDataset/iot23_combined.csv"
df = pd.read_csv(filepath)

In [ ]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,99994,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444670,99995,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444671,99996,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False
1444672,99997,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
del df['Unnamed: 0']

In [ ]:
df['label'].value_counts()

label
PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        197809
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

In [ ]:
Y = pd.get_dummies(df['label']).values

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size= 0.25, random_state=0)


In [ ]:
from sklearn.preprocessing import StandardScaler
st_x= StandardScaler()
x_train= st_x.fit_transform(x_train)
x_test= st_x.transform(x_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )
classifier.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred= classifier.predict(x_test)

In [ ]:
from sklearn.metrics import*

In [ ]:
print("Classifiction Report :")
print(classification_report(y_test, y_pred))

Classifiction Report :
              precision    recall  f1-score   support

           0       0.99      0.89      0.94      4466
           1       0.92      0.99      0.96      5534

   micro avg       0.95      0.95      0.95     10000
   macro avg       0.96      0.94      0.95     10000
weighted avg       0.95      0.95      0.95     10000
 samples avg       0.95      0.95      0.95     10000



In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# Assuming y_test and y_pred are binary matrices of shape (n_samples, n_classes)
mcm = multilabel_confusion_matrix(y_test, y_pred)

# mcm is a 3D array where each matrix corresponds to the confusion matrix of a single class
print(mcm)


[[[5495   39]
  [ 473 3993]]

 [[3993  473]
  [  39 5495]]]


In [ ]:
from matplotlib.colors import ListedColormap
import numpy as nm
import matplotlib.pyplot as mtp
import pandas as pd
x_set, y_set = x_train, y_train
x1, x2 = nm.meshgrid(nm.arange(start = x_set[:, 0].min() - 1, stop = x_set[:, 0].max() + 1, step  =0.01),
nm.arange(start = x_set[:, 1].min() - 1, stop = x_set[:, 1].max() + 1, step = 0.01))
mtp.contourf(x1, x2, classifier.predict(nm.array([x1.ravel(), x2.ravel()]).T).reshape(x1.shape),
alpha = 0.75, cmap = ListedColormap(('red','green' )))
mtp.xlim(x1.min(), x1.max())
mtp.ylim(x2.min(), x2.max())
for i, j in enumerate(nm.unique(y_set)):
    mtp.scatter(x_set[y_set == j, 0], x_set[y_set == j, 1],
        c = ListedColormap(('red', 'green'))(i), label = j)
mtp.title('K-NN Algorithm (Training set)')
mtp.xlabel('Age')
mtp.ylabel('Estimated Salary')
mtp.legend()
mtp.show()

ValueError: X has 2 features, but KNeighborsClassifier is expecting 24 features as input.